In [64]:
from transformers import AutoTokenizer

checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [106]:
from datasets import load_dataset

data = load_dataset("universal_dependencies", "te_mtg")
data

Found cached dataset universal_dependencies (/Users/anishka/.cache/huggingface/datasets/universal_dependencies/te_mtg/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
100%|██████████| 3/3 [00:00<00:00, 306.06it/s]


DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 1051
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 131
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 146
    })
})

In [123]:
def tokenize_and_align_labels(examples):
    #tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    array = []

    for x in (examples["misc"]):
        tokens = []
        for y in x:
            jsonstr = ast.literal_eval(y)
            tokens.append(jsonstr["Translit"])
        array.append(tokens)
    
    tokenized_inputs = tokenizer(array, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"upos"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [124]:
tokenized_dataset = data.map(tokenize_and_align_labels, batched=True, remove_columns=data["train"].column_names)

In [125]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [126]:
from transformers import AutoModelForTokenClassification, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForTokenClassification.from_pretrained(checkpoint)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to us

In [127]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_names = data["train"].features["upos"].feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

label_names = data["train"].features["upos"].feature.names
model = AutoModelForTokenClassification.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to us

In [135]:
training_args = TrainingArguments(
    output_dir="./xlm-roberta-base-te-transliterate",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model('./xlm-roberta-base-te') #For reuse

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f94d97e9af0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 
 33%|███▎      | 66/198 [02:59<05:41,  2.59s/it]









                                                

 33%|███▎      | 66/198 [03:02<05:41,  2.59s/it]



{'eval_loss': 0.5426246523857117, 'eval_runtime': 3.5221, 'eval_samples_per_second': 37.194, 'eval_steps_per_second': 2.555, 'epoch': 1.0}


 67%|██████▋   | 132/198 [06:04<02:51,  2.59s/it]









                                                 

 67%|██████▋   | 132/198 [06:08<02:51,  2.59s/it]



{'eval_loss': 0.48185598850250244, 'eval_runtime': 4.2793, 'eval_samples_per_second': 30.613, 'eval_steps_per_second': 2.103, 'epoch': 2.0}


100%|██████████| 198/198 [10:22<00:00,  3.09s/it]









                                                 

100%|██████████| 198/198 [10:27<00:00,  3.09s/it]


100%|██████████| 198/198 [10:27<00:00,  3.17s/it]


{'eval_loss': 0.46543607115745544, 'eval_runtime': 4.9088, 'eval_samples_per_second': 26.687, 'eval_steps_per_second': 1.833, 'epoch': 3.0}
{'train_runtime': 627.888, 'train_samples_per_second': 5.022, 'train_steps_per_second': 0.315, 'train_loss': 0.4452792080965909, 'epoch': 3.0}


In [136]:
access_token = "XY#################"
from huggingface_hub import login 

login(token=access_token)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/anishka/.cache/huggingface/token
Login successful


In [137]:
trainer.push_to_hub()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('/Users/anishka/Desktop/ccir/codeswitching/./xlm-roberta-base-te-transliterate'). If you really want to do this, do it manually:
 cd /Users/anishka/Desktop/ccir/codeswitching/./xlm-roberta-base-te-transliterate && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.